In [30]:
from os.path import isdir, join
import pickle
import numpy as np
import matplotlib.pyplot as plt
# import rosbag
import glob
from sklearn.model_selection import train_test_split, KFold
import os
import pdb
import math
import torch

In [31]:
pkg_path = '/home/hcalab/Desktop/MYJ/FIT/HumanIntention'
dataset_filename = 'multi_intention_labeled.p'

dataset_filepath = join(pkg_path, 'Dataset', dataset_filename)
with open(dataset_filepath,'rb') as f:
    # data = pickle.load(f)
    
    # for python3
    u = pickle._Unpickler( f )
    u.encoding = 'latin1'
    data = u.load()

    

In [32]:
xtrain, xtest, ytrain, ytest = [], [], [], []
for intention in [0, 1, 2]:
    intention_label = [intention for i in range(len(data[intention]))]
    xtraini, xtesti, ytraini, ytesti = train_test_split(data[intention], intention_label, test_size=5, random_state=0)
    xtrain += xtraini
    ytrain += ytraini
    xtest += xtesti
    ytest += ytesti

In [33]:
def process_data(raw_x, raw_y, obs_seq_len, pred_seq_len, skip=2):
    seq_len = obs_seq_len+pred_seq_len
    xobs, xpred, yintention = [], [], []
    for x, y in zip(raw_x, raw_y):
        n_seq = math.floor((len(x)-seq_len)/skip)+1
        idx_range = range(0, n_seq*skip, skip)
        for idx in idx_range:
            seq_data = x[idx:idx+seq_len]# (seq_len, 2)
            obs_seq_data, pred_seq_data = seq_data[:obs_seq_len], seq_data[obs_seq_len:]
            xobs.append(obs_seq_data)
            xpred.append(pred_seq_data)
            yintention.append(y)
    xobs = torch.stack(xobs, dim=0) # (N, obs_seq_len, 2)
    xpred = torch.stack(xpred, dim=0) # (N, pred_seq_len, 2)
    yintention = torch.tensor(yintention).unsqueeze(1).type(torch.float) # (N, 1)
    return xobs, xpred, yintention

skip = 2
obs_seq_len, pred_seq_len = 4, 6

xobs_train, xpred_train, yintention_train = process_data(xtrain, ytrain, obs_seq_len, pred_seq_len, skip=2)
xobs_test, xpred_test, yintention_test = process_data(xtest, ytest, obs_seq_len, pred_seq_len, skip=2)

In [34]:
torch.save({"xobs_train": xobs_train, "xpred_train": xpred_train, "yintention_train": yintention_train,\
            "xobs_test": xobs_test, "xpred_test": xpred_test, "yintention_test": yintention_test, \
            "obs_seq_len": obs_seq_len, "pred_seq_len": pred_seq_len, "skip": skip}, 'short_prediction_multi_intention_data.pt')

In [35]:
print(sum(yintention_train==0))
print(sum(yintention_train==1))
print(sum(yintention_train==2))

tensor([386])
tensor([520])
tensor([351])


In [36]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, validation_index in kf.split(xobs_train):
#     print("TRAIN:", train_index, "VAL:", validation_index)
    print(xobs_train[train_index][20])
    print(sum(yintention_train[validation_index]==0))
    print(sum(yintention_train[validation_index]==1))
    print(sum(yintention_train[validation_index]==2))
    print()
    break

tensor([[-0.8620,  0.2688,  0.1217,  0.0427],
        [-0.8629,  0.2476,  0.1302,  0.0431],
        [-0.8589,  0.2395,  0.1468,  0.0409],
        [-0.8612,  0.2223,  0.1494,  0.0772]])
tensor([64])
tensor([110])
tensor([78])



In [37]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
kf.get_n_splits()
# print(kf)


5

In [38]:
print(ytrain)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]


In [39]:
for train_index, validation_index in kf.split(xtrain, ytrain):
    print("TRAIN:", train_index, "VAL:", validation_index)
    print('TRAIN intentions:', np.array(ytrain)[train_index])
    print('VAL intentions:', np.array(ytrain)[validation_index])
    

TRAIN: [ 0  1  3  4  6  7  8  9 10 12 13 14 15 17 18 20 21 22 23 24] VAL: [ 2  5 11 16 19]
TRAIN intentions: [0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2]
VAL intentions: [0 0 1 1 2]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 18 19 20 21] VAL: [14 17 22 23 24]
TRAIN intentions: [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2]
VAL intentions: [1 1 2 2 2]
TRAIN: [ 0  2  3  4  5  7  9 11 12 14 15 16 17 18 19 20 21 22 23 24] VAL: [ 1  6  8 10 13]
TRAIN intentions: [0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2]
VAL intentions: [0 0 1 1 1]
TRAIN: [ 0  1  2  3  5  6  8 10 11 12 13 14 15 16 17 19 21 22 23 24] VAL: [ 4  7  9 18 20]
TRAIN intentions: [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2]
VAL intentions: [0 1 1 2 2]
TRAIN: [ 1  2  4  5  6  7  8  9 10 11 13 14 16 17 18 19 20 22 23 24] VAL: [ 0  3 12 15 21]
TRAIN intentions: [0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2]
VAL intentions: [0 0 1 1 2]
